In [1]:
import os
import requests
import dotenv
import time
import numpy as np
import pandas as pd
import alpaca_trade_api as alpaca_api
import krakenex
import pykrakenapi
import pandas_montecarlo

%matplotlib inline

In [2]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

In [3]:
dotenv.load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv('alpaca_api_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')
kraken_api_key = os.getenv('kraken_api_key')
kraken_secret_key = os.getenv('kraken_secret_key')

print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(kraken_api_key))
print(type(kraken_secret_key))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [5]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

faangmula_tickers =  ['AAPL',  'ABNB', 'AMZN', 'FB', 'GOOG', 'LYFT', 'MSFT', 'NFLX', 'UBER']
start = pd.Timestamp('2015-12-01', tz='America/New_York').isoformat()

faangmula_original = alpaca.get_bars(faangmula_tickers, alpaca_api.TimeFrame.Day, start = start).df
faangmula_original

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,118.73,118.810,116.860,117.34,34852374,187129,117.756760,AAPL
2015-12-02 05:00:00+00:00,117.00,118.110,116.080,116.28,33385643,180616,117.151198,AAPL
2015-12-03 05:00:00+00:00,116.55,116.790,114.220,115.20,41560785,245330,115.434888,AAPL
2015-12-04 05:00:00+00:00,115.20,119.250,115.110,119.03,57776977,307788,118.187290,AAPL
2015-12-07 05:00:00+00:00,118.98,119.860,117.810,118.28,32080754,190809,118.509111,AAPL
...,...,...,...,...,...,...,...,...
2022-04-04 04:00:00+00:00,36.40,36.930,36.215,36.51,20197208,146275,36.621379,UBER
2022-04-05 04:00:00+00:00,36.43,36.610,34.720,34.96,20987999,153337,35.188058,UBER
2022-04-06 04:00:00+00:00,33.82,34.000,32.550,33.41,29163174,203021,33.287397,UBER


In [6]:
faangmula = faangmula_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis = 1).pivot(columns = 'symbol')
faangmula.index = faangmula.index.date
faangmula.columns = faangmula_tickers
faangmula

,AAPL,ABNB,AMZN,FB,GOOG,LYFT,MSFT,NFLX,UBER
2015-12-01,117.34,NaN,679.06,107.12,767.04,NaN,55.22,125.37,NaN
2015-12-02,116.28,NaN,676.01,106.07,762.38,NaN,55.21,128.93,NaN
2015-12-03,115.20,NaN,666.25,104.38,752.54,NaN,54.20,126.81,NaN
2015-12-04,119.03,NaN,672.64,106.18,766.81,NaN,55.91,130.93,NaN
2015-12-07,118.28,NaN,669.83,105.61,763.25,NaN,55.81,125.36,NaN
...,...,...,...,...,...,...,...,...,...
2022-04-04,178.44,177.02,3366.93,233.89,2872.85,38.96,314.97,391.50,36.51
2022-04-05,175.06,171.21,3281.10,231.84,2821.26,37.15,310.88,380.15,34.96
2022-04-06,171.59,164.66,3175.12,223.30,2743.52,36.08,299.50,368.35,33.41
2022-04-07,172.14,165.91,3155.69,222.95,2729.30,34.61,301.37,362.15,32.27


In [7]:
# Market capitalization data in trillions of dollars, sourced from Trading View on April 6, 2022.

faangmula_caps_list = [2.804, 0.105855, 1.616, 0.60781, 1.81, 0.012576, 2.245, 0.163534, 0.065299]
faangmula_cap_portfolio = pd.DataFrame({'tickers':faangmula_tickers, 'market cap weights':[market_cap / sum(faangmula_caps_list) for market_cap in faangmula_caps_list]}).set_index('tickers')
faangmula_cap_portfolio

,market cap weights
tickers,
AAPL,0.297347
ABNB,0.011225
AMZN,0.171367
FB,0.064454
GOOG,0.191939
LYFT,0.001334
MSFT,0.238068
NFLX,0.017342
UBER,0.006925


In [8]:
faangmula_cap_daily_returns = faangmula.pct_change().fillna(0.0).dot(faangmula_cap_portfolio)
faangmula_cap_daily_returns.columns = ['faangmula index']
faangmula_cap_daily_returns

,faangmula index
2015-12-01,0.000000
2015-12-02,-0.004804
2015-12-03,-0.013380
2015-12-04,0.024355
2015-12-07,-0.004990
...,...
2022-04-04,0.024182
2022-04-05,-0.018331
2022-04-06,-0.029120
2022-04-07,-0.000202


In [9]:
random_faangmula_portfolios = random_portfolios_generator(faangmula_tickers, 100)
random_faangmula_portfolios

/usr/local/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.


,ticker,random weights 1,random weights 2,random weights 3,random weights 4,random weights 5,random weights 6,random weights 7,random weights 8,random weights 9,...,random weights 91,random weights 92,random weights 93,random weights 94,random weights 95,random weights 96,random weights 97,random weights 98,random weights 99,random weights 100
0,AAPL,0.054504,0.113142,0.026786,0.167568,0.185080,0.046399,0.257347,0.019832,0.169856,...,0.099336,0.150151,0.008038,0.138804,0.146064,0.177981,0.043765,0.097738,0.187774,0.144868
1,ABNB,0.158375,0.146224,0.049038,0.002430,0.103838,0.014071,0.015727,0.209503,0.117484,...,0.303024,0.109556,0.014289,0.112974,0.116592,0.022193,0.141588,0.017254,0.043000,0.090806
2,AMZN,0.050132,0.167672,0.187702,0.046602,0.193441,0.148090,0.013004,0.006785,0.213181,...,0.077607,0.017698,0.096983,0.108846,0.142408,0.140237,0.062449,0.141464,0.146475,0.124119
3,FB,0.061213,0.114066,0.176610,0.145402,0.069103,0.167044,0.147443,0.119971,0.020181,...,0.127775,0.139329,0.242994,0.016699,0.177849,0.129999,0.086858,0.115869,0.163872,0.175302
4,GOOG,0.199541,0.084993,0.043391,0.173525,0.005027,0.039978,0.008111,0.029691,0.173048,...,0.060410,0.001492,0.102076,0.193193,0.055453,0.018787,0.156530,0.147568,0.096893,0.120746
5,LYFT,0.124650,0.060381,0.075762,0.168014,0.132544,0.180652,0.155624,0.242553,0.064552,...,0.044190,0.102625,0.145103,0.039003,0.172355,0.135917,0.170719,0.066744,0.169964,0.056274
6,MSFT,0.032992,0.043736,0.162320,0.022838,0.149598,0.158648,0.158730,0.139308,0.172002,...,0.104347,0.051476,0.096225,0.138196,0.051178,0.137072,0.131011,0.139647,0.152858,0.084278
7,NFLX,0.150955,0.106124,0.035338,0.053284,0.038316,0.115464,0.193196,0.021972,0.019494,...,0.067823,0.149905,0.177434,0.088683,0.087631,0.159740,0.136069,0.135395,0.015113,0.191751
8,UBER,0.167637,0.163661,0.243053,0.220336,0.123053,0.129654,0.050819,0.210385,0.050202,...,0.115489,0.277766,0.116857,0.163602,0.050470,0.078074,0.071012,0.138323,0.024052,0.011856


In [10]:
random_faangmula_daily_returns_list = []
for n in range(0, (len(random_faangmula_portfolios.columns) - 1)):
        random_faangmula_daily_returns_list.append(faangmula.pct_change().fillna(0.0).dot(list(random_faangmula_portfolios.loc[:, f'random weights {n + 1}'])))

random_faangmula_daily_returns = pd.concat(random_faangmula_daily_returns_list, axis = 'columns', join = 'inner')

for n in range(0, (len(random_faangmula_portfolios.columns) - 1)):
        random_faangmula_daily_returns = random_faangmula_daily_returns.rename(columns = {n: f'random faangmula portfolio {n + 1}'})

random_faangmula_daily_returns

,random faangmula portfolio 1,random faangmula portfolio 2,random faangmula portfolio 3,random faangmula portfolio 4,random faangmula portfolio 5,random faangmula portfolio 6,random faangmula portfolio 7,random faangmula portfolio 8,random faangmula portfolio 9,random faangmula portfolio 10,...,random faangmula portfolio 91,random faangmula portfolio 92,random faangmula portfolio 93,random faangmula portfolio 94,random faangmula portfolio 95,random faangmula portfolio 96,random faangmula portfolio 97,random faangmula portfolio 98,random faangmula portfolio 99,random faangmula portfolio 100
2015-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-02,0.001751,-0.000404,-0.002106,-0.002694,-0.002188,0.000285,0.001580,-0.000967,-0.003219,-0.003857,...,-0.000958,0.001437,0.001511,-0.000587,-0.001560,0.000885,0.001362,0.000269,-0.004148,0.001112
2015-12-03,-0.007867,-0.008931,-0.009883,-0.008079,-0.009044,-0.010547,-0.011112,-0.005487,-0.010678,-0.010889,...,-0.007883,-0.007296,-0.011342,-0.009607,-0.009339,-0.011126,-0.009346,-0.011482,-0.010765,-0.012184
2015-12-04,0.013078,0.013776,0.012828,0.014268,0.015260,0.015358,0.022662,0.008465,0.017381,0.017971,...,0.012892,0.014087,0.016124,0.016851,0.014802,0.019375,0.015074,0.018207,0.017625,0.020208
2015-12-07,-0.008289,-0.007013,-0.003896,-0.005144,-0.004266,-0.007189,-0.011008,-0.002119,-0.004010,-0.004406,...,-0.004988,-0.008244,-0.009955,-0.006336,-0.006547,-0.009533,-0.007753,-0.008524,-0.004041,-0.011241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.028140,0.027924,0.026360,0.027540,0.026732,0.030569,0.031830,0.026862,0.024618,0.025692,...,0.026484,0.028399,0.032841,0.024431,0.031106,0.030984,0.029379,0.028234,0.028680,0.031814
2022-04-05,-0.029341,-0.027141,-0.026200,-0.027833,-0.026976,-0.026602,-0.024210,-0.031656,-0.023721,-0.023760,...,-0.026288,-0.029713,-0.025152,-0.025851,-0.026133,-0.025191,-0.027232,-0.024401,-0.023219,-0.022364
2022-04-06,-0.033428,-0.033687,-0.036168,-0.032155,-0.032512,-0.033839,-0.030320,-0.035960,-0.031245,-0.031708,...,-0.034758,-0.034721,-0.033992,-0.032716,-0.031830,-0.031565,-0.032899,-0.032871,-0.030831,-0.031323
2022-04-07,-0.013193,-0.009730,-0.012154,-0.015975,-0.009246,-0.013857,-0.009744,-0.015290,-0.004394,-0.007615,...,-0.004601,-0.014882,-0.013654,-0.008192,-0.009984,-0.010455,-0.010965,-0.010214,-0.007773,-0.005905


In [11]:
faangmula_daily_returns = pd.concat([faangmula_cap_daily_returns, random_faangmula_daily_returns], axis = 1, join = 'inner')
faangmula_daily_returns

,faangmula index,random faangmula portfolio 1,random faangmula portfolio 2,random faangmula portfolio 3,random faangmula portfolio 4,random faangmula portfolio 5,random faangmula portfolio 6,random faangmula portfolio 7,random faangmula portfolio 8,random faangmula portfolio 9,...,random faangmula portfolio 91,random faangmula portfolio 92,random faangmula portfolio 93,random faangmula portfolio 94,random faangmula portfolio 95,random faangmula portfolio 96,random faangmula portfolio 97,random faangmula portfolio 98,random faangmula portfolio 99,random faangmula portfolio 100
2015-12-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-12-02,-0.004804,0.001751,-0.000404,-0.002106,-0.002694,-0.002188,0.000285,0.001580,-0.000967,-0.003219,...,-0.000958,0.001437,0.001511,-0.000587,-0.001560,0.000885,0.001362,0.000269,-0.004148,0.001112
2015-12-03,-0.013380,-0.007867,-0.008931,-0.009883,-0.008079,-0.009044,-0.010547,-0.011112,-0.005487,-0.010678,...,-0.007883,-0.007296,-0.011342,-0.009607,-0.009339,-0.011126,-0.009346,-0.011482,-0.010765,-0.012184
2015-12-04,0.024355,0.013078,0.013776,0.012828,0.014268,0.015260,0.015358,0.022662,0.008465,0.017381,...,0.012892,0.014087,0.016124,0.016851,0.014802,0.019375,0.015074,0.018207,0.017625,0.020208
2015-12-07,-0.004990,-0.008289,-0.007013,-0.003896,-0.005144,-0.004266,-0.007189,-0.011008,-0.002119,-0.004010,...,-0.004988,-0.008244,-0.009955,-0.006336,-0.006547,-0.009533,-0.007753,-0.008524,-0.004041,-0.011241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.024182,0.028140,0.027924,0.026360,0.027540,0.026732,0.030569,0.031830,0.026862,0.024618,...,0.026484,0.028399,0.032841,0.024431,0.031106,0.030984,0.029379,0.028234,0.028680,0.031814
2022-04-05,-0.018331,-0.029341,-0.027141,-0.026200,-0.027833,-0.026976,-0.026602,-0.024210,-0.031656,-0.023721,...,-0.026288,-0.029713,-0.025152,-0.025851,-0.026133,-0.025191,-0.027232,-0.024401,-0.023219,-0.022364
2022-04-06,-0.029120,-0.033428,-0.033687,-0.036168,-0.032155,-0.032512,-0.033839,-0.030320,-0.035960,-0.031245,...,-0.034758,-0.034721,-0.033992,-0.032716,-0.031830,-0.031565,-0.032899,-0.032871,-0.030831,-0.031323
2022-04-07,-0.000202,-0.013193,-0.009730,-0.012154,-0.015975,-0.009246,-0.013857,-0.009744,-0.015290,-0.004394,...,-0.004601,-0.014882,-0.013654,-0.008192,-0.009984,-0.010455,-0.010965,-0.010214,-0.007773,-0.005905


In [12]:
faangmula_correlations = faangmula_daily_returns.corr()
faangmula_correlations

,faangmula index,random faangmula portfolio 1,random faangmula portfolio 2,random faangmula portfolio 3,random faangmula portfolio 4,random faangmula portfolio 5,random faangmula portfolio 6,random faangmula portfolio 7,random faangmula portfolio 8,random faangmula portfolio 9,...,random faangmula portfolio 91,random faangmula portfolio 92,random faangmula portfolio 93,random faangmula portfolio 94,random faangmula portfolio 95,random faangmula portfolio 96,random faangmula portfolio 97,random faangmula portfolio 98,random faangmula portfolio 99,random faangmula portfolio 100
faangmula index,1.000000,0.778178,0.865242,0.804806,0.812557,0.892530,0.820549,0.917358,0.606303,0.960402,...,0.828578,0.769199,0.807996,0.909321,0.876817,0.913251,0.819180,0.903033,0.930460,0.929874
random faangmula portfolio 1,0.778178,1.000000,0.976340,0.962780,0.961300,0.948762,0.972594,0.917099,0.934093,0.902857,...,0.952250,0.975325,0.962472,0.963614,0.961728,0.943508,0.990171,0.954904,0.919392,0.899333
random faangmula portfolio 2,0.865242,0.976340,1.000000,0.973102,0.953296,0.978656,0.973466,0.947729,0.880734,0.955449,...,0.973596,0.965335,0.963901,0.984891,0.984569,0.975954,0.976678,0.980927,0.957913,0.950869
random faangmula portfolio 3,0.804806,0.962780,0.973102,1.000000,0.961005,0.955698,0.985258,0.900060,0.913896,0.912992,...,0.931669,0.956775,0.970187,0.954499,0.957892,0.942772,0.963002,0.964861,0.937632,0.893254
random faangmula portfolio 4,0.812557,0.961300,0.953296,0.961005,1.000000,0.964885,0.967482,0.935723,0.929432,0.902248,...,0.906378,0.976986,0.938345,0.950839,0.962744,0.946467,0.953820,0.942927,0.952033,0.868981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
random faangmula portfolio 96,0.913251,0.943508,0.975954,0.942772,0.946467,0.976353,0.970443,0.989222,0.824802,0.961696,...,0.925605,0.941844,0.960437,0.975353,0.983236,1.000000,0.961892,0.987367,0.977435,0.972151
random faangmula portfolio 97,0.819180,0.990171,0.976678,0.963002,0.953820,0.958654,0.983646,0.936424,0.915940,0.929534,...,0.954272,0.955398,0.974751,0.968073,0.977154,0.961892,1.000000,0.968996,0.947690,0.927791
random faangmula portfolio 98,0.903033,0.954904,0.980927,0.964861,0.942927,0.961857,0.977491,0.961938,0.825794,0.961606,...,0.927705,0.935053,0.977144,0.983623,0.971072,0.987367,0.968996,1.000000,0.964755,0.971585
random faangmula portfolio 99,0.930460,0.919392,0.957913,0.937632,0.952033,0.981373,0.955534,0.967693,0.837542,0.975315,...,0.924779,0.914708,0.932261,0.964618,0.983960,0.977435,0.947690,0.964755,1.000000,0.943550


In [13]:
low_correlation_faangmula_original = faangmula_correlations.sort_values('faangmula index')['faangmula index'].head(5).reset_index().rename({'index': 'portfolio', 'faangmula index': 'faangmula index correlation'}, axis = 1)
low_correlation_faangmula_original

,portfolio,faangmula index correlation
0,random faangmula portfolio 8,0.606303
1,random faangmula portfolio 48,0.741951
2,random faangmula portfolio 17,0.743446
3,random faangmula portfolio 54,0.744052
4,random faangmula portfolio 59,0.760170


In [14]:
variances = []
covariances = []
for portfolio in low_correlation_faangmula_original['portfolio']:
    variance = faangmula_daily_returns[f'{portfolio}'].var()
    variances.append(variance)
    covariance = faangmula_daily_returns[f'{portfolio}'].cov(faangmula_daily_returns['faangmula index'])
    covariances.append(covariance)
low_correlation_faangmula_withvar = pd.concat([low_correlation_faangmula_original, pd.DataFrame(variances)], axis = 1).rename({0: 'portfolio variance'}, axis = 1)
low_correlation_faangmula_withcov = pd.concat([low_correlation_faangmula_withvar, pd.DataFrame(covariances)], axis = 1).rename({0: 'faangmula index covariance'}, axis = 1)
low_correlation_faangmula_withcov

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance
0,random faangmula portfolio 8,0.606303,0.000244,0.000154
1,random faangmula portfolio 48,0.741951,0.000191,0.000167
2,random faangmula portfolio 17,0.743446,0.000198,0.000171
3,random faangmula portfolio 54,0.744052,0.000233,0.000185
4,random faangmula portfolio 59,0.760170,0.000190,0.000171


In [15]:
betas = []
for portfolio in low_correlation_faangmula_withcov['portfolio']:
    beta = low_correlation_faangmula_withcov.set_index('portfolio').at[f'{portfolio}', 'faangmula index covariance'] / low_correlation_faangmula_withcov.set_index('portfolio').at[f'{portfolio}', 'portfolio variance']
    betas.append(beta)
low_correlation_faangmula_withbeta = pd.concat([low_correlation_faangmula_withcov, pd.DataFrame(betas)], axis = 1).rename({0: 'faangmula index beta'}, axis = 1)
low_correlation_faangmula_withbeta

,portfolio,faangmula index correlation,portfolio variance,faangmula index covariance,faangmula index beta
0,random faangmula portfolio 8,0.606303,0.000244,0.000154,0.632415
1,random faangmula portfolio 48,0.741951,0.000191,0.000167,0.875120
2,random faangmula portfolio 17,0.743446,0.000198,0.000171,0.860840
3,random faangmula portfolio 54,0.744052,0.000233,0.000185,0.794572
4,random faangmula portfolio 59,0.760170,0.000190,0.000171,0.899933
